# Relevamiento de carga de datos sospechosa en las elecciones de Provincia de Buenos Aires

Este análisis es mi aporte para hacer el trabajo de **chequeo manual de telegramas de las PASO 2017** en la provincia de Buenos Aires de una más eficientemente, detectando aquellas resultados estádisticamente raros (no quiere decir que todas las mesas listadas estén incorrectas). 

Para esto realizé un programa que obtiene los datos digitalizados de cada una de las mesas y los estructuré en una base de datos para hacer agregaciones y consultas. 

Por ejemplo, basado en la idea de que los resultados en una mesa son similares en un circuito, se pueden detectar aquellas mesas que tienen resultados muy distintos. 

Las tablas resultantes muestran algunos datos e incluyen el link para ver el telegrama en el sitio oficial para que revisemos entre tod@s. 

Todo el [código fuente está disponible en github](http://github/mgaitan/mesas_ba/)

**Gracias por difundir**

- Martín Gaitán [`@tin_nqn_`](https://twitter.com/tin_nqn_)



<div class="alert alert-info" role="alert">Si querés podés <a href="#resultados">ir a los resultados</div>



### Generación de base de datos

Utilizo el ORM de Django para modelar la base de datos y luego importo los CSVs del [crawler del escrutinio provisorio](https://github.com/mgaitan/mesas_ba/tree/master/mesas_ba), mesa por mesa. 


In [1]:
!pip install django-orm-magic

  Running setup.py bdist_wheel for django-orm-magic ... done
  Stored in directory: /home/tin/.cache/pip/wheels/6e/45/52/31f3c132bc0fb005bebab9482133db4a9a5cadb98e7684551b
Successfully built django-orm-magic


In [1]:
%load_ext django_orm_magic

In [2]:
%%django_orm
from django.db import models
from django.db.models import Sum

class Seccion(models.Model):
    numero = models.CharField(max_length=10)
    nombre = models.CharField(max_length=100)
    
    def __str__(self):
        return f'{self.numero} - {self.nombre}'

class Circuito(models.Model):
    numero = models.CharField(max_length=10)
    seccion = models.ForeignKey('Seccion')

    def __str__(self):
        return u"Circuito %s (%s)" % (self.numero, self.seccion)
    
class Mesa(models.Model):
    numero = models.CharField(max_length=10)
    circuito = models.ForeignKey('Circuito')
    fake_id = models.CharField(max_length=100, unique=True)
    url = models.CharField(max_length=200)
    
    class Meta:
        unique_together = ('circuito', 'numero')
    
    @property
    def computados(self):
        return self.resultados.aggregate(Sum('votos'))['votos__sum']

    def __str__(self):
        return u"Mesa %s (%s)" % (self.numero, self.circuito)
    
    @property
    def absolute_url(self):
        return f'http://www.resultados.gob.ar/99/resu/content/telegramas/{self.url}'
    

class Partido(models.Model):
    nombre = models.CharField(max_length=100, unique=True)
        
    def __str__(self):
        return self.nombre
    
    
class Opcion(models.Model):
    nombre = models.CharField(max_length=100, unique=True)
    partido = models.ForeignKey('Partido', null=True)    # blanco / nulo / etc. 
    
    def __str__(self):
        return self.nombre
    
class VotoMesa(models.Model):
    mesa = models.ForeignKey('Mesa', related_name='resultados')
    categoria = models.CharField(max_length=20, choices=(('dip', 'Diputados'), ('sen', 'Senadores')))
    opcion = models.ForeignKey('Opcion')
    votos = models.IntegerField()
    
    def __str__(self):
        return u"%s: %s -> %d" % (self.get_categoria_display(), self.opcion, self.votos)
    
    
    class Meta:
        unique_together = ('mesa', 'opcion', 'categoria')

## Importación de datos a los modelos

La [base de datos está disponible](http://lab.nqnwebs.com/descargas/db.sqlite3) (Md5sum: `0e5d3f0c92ba8d18fc19b7a3ed742266`) Teniendo esta base **no hace falta que repitas el scraping e importes los datos** Podés salterte esto.

In [5]:
from csv import DictReader

In [6]:
Seccion.objects.all()

<QuerySet []>

Importo secciones, circuitos y mesas de los distintos archivos

In [7]:
for seccion in DictReader(open('secciones.csv')):
    Seccion.objects.create(**seccion)
    

In [8]:
secciones = {s.numero: s for s in Seccion.objects.all()}
circuitos = []
for circuito in DictReader(open('circuitos.csv')):
    circuito['seccion'] = secciones[circuito['seccion']] 
    circuitos.append(Circuito(**circuito))
Circuito.objects.bulk_create(circuitos);

In [9]:
circuitos = {c.numero: c for c in Circuito.objects.all()}
mesas = []
for mesa in DictReader(open('mesas.csv')):
    mesa['circuito'] = circuitos[mesa['circuito']]
    mesas.append(Mesa(**mesa))
Mesa.objects.bulk_create(mesas);

El archivo `resultados.csv` pesa casi 100mb y tiene una fila por cada mesa-opcion, para senadores y diputados. 
Para cargarlo más rapido (sino puede tardar muchisimo) hice unas pequeñas optimizaciones. 

- Importo partidos y listas que existan primero y armo un diccionario para no tener que hacer consultas luego
- Hago "chunks" e inserto varias filas de resultados a la vez. 

Con esto tarda unos 5' en cargar. 

In [10]:
from itertools import zip_longest

def grouper(n, iterable, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(fillvalue=fillvalue, *args)

In [15]:
import pandas as pd
partidos_listas = pd.read_csv('resultados.csv', usecols=['partido','lista'])
partidos_listas.drop_duplicates(inplace=True)


lista  \
0                            NaN   
2                            NaN   
4                            NaN   
6                            NaN   
8                     ROJO PUNZO   
10              CELESTE Y BLANCA   
12              FEDERALISMO PURO   
14                   ALTERNATIVA   
31                       NARANJA   
33                 INDEPENDENCIA   
35                      LIBERTAD   
36                     SEGURIDAD   
37                      UNIDAD 1   
39                          TOBA   
41                         VAMOS   
42          COMPROMISO CIUDADANO   
44   UN PAIS UNIDO HOY Y SIEMPRE   
58                       LISTA A   
60            CELESTE Y BLANCA U   
62       UNIDAD DE LA  IZQUIERDA   
64                    CONDUCCION   
66                     ESPERANZA   
67            ORDEN  Y SEGURIDAD   
68                   SOLIDARIDAD   
69                      PIONEROS   
126                    SOBERANIA   
127   MEMORIA, VERDAD Y JUSTICIA   
128                 CONVICCIONES   
130             CAMBIANDO JUNTOS   
132                      CUMPLIR   
134                      LISTA 1   

                                           partido  
0                                            nulos  
2                                          blancos  
4                                       recurridos  
6                                      impuganados  
8                                          FEDERAL  
10                                         FEDERAL  
12                                         FEDERAL  
14                                         FEDERAL  
31                                       HUMANISTA  
33                               DEL CAMPO POPULAR  
35                               DEL CAMPO POPULAR  
36                               DEL CAMPO POPULAR  
37   MOVIMIENTO AMPLIO DE TRABAJADORES Y JUBILADOS  
39                          TODOS POR BUENOS AIRES  
41                                   PATRIA GRANDE  
42             MOVIMIENTO ORGANIZACION DEMOCRATICA  
44                                           1PAIS  
58                     FRENTE SOCIALISTA Y POPULAR  
60                                UNIDAD CIUDADANA  
62           IZQUIERDA AL FRENTE POR EL SOCIALISMO  
64                 FRENTE PATRIOTA BANDERA VECINAL  
66                 FRENTE PATRIOTA BANDERA VECINAL  
67                 FRENTE PATRIOTA BANDERA VECINAL  
68                 FRENTE PATRIOTA BANDERA VECINAL  
69                 FRENTE PATRIOTA BANDERA VECINAL  
126                FRENTE PATRIOTA BANDERA VECINAL  
127   ENCUENTRO POPULAR POR TIERRA TECHO Y TRABAJO  
128                                           CREO  
130                         CAMBIEMOS BUENOS AIRES  
132                           FRENTE JUSTICIALISTA  
134      FRENTE DE IZQUIERDA Y DE LOS TRABAJADORES

In [31]:
partidos_listas.fillna('', inplace=True)
partidos_listas

lista  \
0                                  
2                                  
4                                  
6                                  
8                     ROJO PUNZO   
10              CELESTE Y BLANCA   
12              FEDERALISMO PURO   
14                   ALTERNATIVA   
31                       NARANJA   
33                 INDEPENDENCIA   
35                      LIBERTAD   
36                     SEGURIDAD   
37                      UNIDAD 1   
39                          TOBA   
41                         VAMOS   
42          COMPROMISO CIUDADANO   
44   UN PAIS UNIDO HOY Y SIEMPRE   
58                       LISTA A   
60            CELESTE Y BLANCA U   
62       UNIDAD DE LA  IZQUIERDA   
64                    CONDUCCION   
66                     ESPERANZA   
67            ORDEN  Y SEGURIDAD   
68                   SOLIDARIDAD   
69                      PIONEROS   
126                    SOBERANIA   
127   MEMORIA, VERDAD Y JUSTICIA   
128                 CONVICCIONES   
130             CAMBIANDO JUNTOS   
132                      CUMPLIR   
134                      LISTA 1   

                                           partido  
0                                            nulos  
2                                          blancos  
4                                       recurridos  
6                                      impuganados  
8                                          FEDERAL  
10                                         FEDERAL  
12                                         FEDERAL  
14                                         FEDERAL  
31                                       HUMANISTA  
33                               DEL CAMPO POPULAR  
35                               DEL CAMPO POPULAR  
36                               DEL CAMPO POPULAR  
37   MOVIMIENTO AMPLIO DE TRABAJADORES Y JUBILADOS  
39                          TODOS POR BUENOS AIRES  
41                                   PATRIA GRANDE  
42             MOVIMIENTO ORGANIZACION DEMOCRATICA  
44                                           1PAIS  
58                     FRENTE SOCIALISTA Y POPULAR  
60                                UNIDAD CIUDADANA  
62           IZQUIERDA AL FRENTE POR EL SOCIALISMO  
64                 FRENTE PATRIOTA BANDERA VECINAL  
66                 FRENTE PATRIOTA BANDERA VECINAL  
67                 FRENTE PATRIOTA BANDERA VECINAL  
68                 FRENTE PATRIOTA BANDERA VECINAL  
69                 FRENTE PATRIOTA BANDERA VECINAL  
126                FRENTE PATRIOTA BANDERA VECINAL  
127   ENCUENTRO POPULAR POR TIERRA TECHO Y TRABAJO  
128                                           CREO  
130                         CAMBIEMOS BUENOS AIRES  
132                           FRENTE JUSTICIALISTA  
134      FRENTE DE IZQUIERDA Y DE LOS TRABAJADORES

In [32]:
Partido.objects.all().delete()

(0, {})

In [33]:
for _, row in partidos_listas.iterrows():
    if not row.lista:
        Opcion.objects.create(nombre=row.partido)
    else:
        partido, _ = Partido.objects.get_or_create(nombre=row.partido)
        Opcion.objects.create(nombre=row.lista, partido=partido)

In [34]:
opciones = {o.nombre: o for o in Opcion.objects.all()}
opciones

{'ALTERNATIVA': <Opcion: ALTERNATIVA>,
 'CAMBIANDO JUNTOS': <Opcion: CAMBIANDO JUNTOS>,
 'CELESTE Y BLANCA': <Opcion: CELESTE Y BLANCA>,
 'CELESTE Y BLANCA U': <Opcion: CELESTE Y BLANCA U>,
 'COMPROMISO CIUDADANO': <Opcion: COMPROMISO CIUDADANO>,
 'CONDUCCION': <Opcion: CONDUCCION>,
 'CONVICCIONES': <Opcion: CONVICCIONES>,
 'CUMPLIR': <Opcion: CUMPLIR>,
 'ESPERANZA': <Opcion: ESPERANZA>,
 'FEDERALISMO PURO': <Opcion: FEDERALISMO PURO>,
 'INDEPENDENCIA': <Opcion: INDEPENDENCIA>,
 'LIBERTAD': <Opcion: LIBERTAD>,
 'LISTA 1': <Opcion: LISTA 1>,
 'LISTA A': <Opcion: LISTA A>,
 'MEMORIA, VERDAD Y JUSTICIA': <Opcion: MEMORIA, VERDAD Y JUSTICIA>,
 'NARANJA': <Opcion: NARANJA>,
 'ORDEN  Y SEGURIDAD': <Opcion: ORDEN  Y SEGURIDAD>,
 'PIONEROS': <Opcion: PIONEROS>,
 'ROJO PUNZO': <Opcion: ROJO PUNZO>,
 'SEGURIDAD': <Opcion: SEGURIDAD>,
 'SOBERANIA': <Opcion: SOBERANIA>,
 'SOLIDARIDAD': <Opcion: SOLIDARIDAD>,
 'TOBA': <Opcion: TOBA>,
 'UN PAIS UNIDO HOY Y SIEMPRE': <Opcion: UN PAIS UNIDO HOY Y SIEM

In [36]:
mesas = {m.fake_id: m for m in Mesa.objects.all()}

In [ ]:
for chunk in grouper(250, DictReader(open('resultados.csv'))):
    resultados = []
    for resultado in chunk:
        if not resultado:
            continue
        mesa = mesas[resultado['mesa']]
        lista = resultado['lista'] or resultado['partido']
        opcion = opciones[lista]
        try:
            votos = int(resultado['votos'])
        except ValueError:
            print('**votos no validos', resultado)
            continue
        
        resultados.append(VotoMesa(mesa=mesa, 
                               opcion=opcion, 
                               votos=votos, 
                               categoria=resultado['categoria']))
    VotoMesa.objects.bulk_create(resultados)

<div id="resultados"></div>

## RESULTADOS 



Teniendo la base de datos puedo hacer cualquier tipo de consulta, por mesa, por circuito, totales, etc. 

Primero calculemos los porcentajes totales para diputados y senadores

In [5]:
UC = Opcion.objects.filter(partido__nombre='UNIDAD CIUDADANA')[0]
UC, UC.partido

(<Opcion: CELESTE Y BLANCA U>, <Partido: UNIDAD CIUDADANA>)

In [6]:
CAMBIEMOS = Opcion.objects.get(id=32)
CAMBIEMOS, CAMBIEMOS.partido

(<Opcion: CAMBIANDO JUNTOS>, <Partido: CAMBIEMOS BUENOS AIRES>)

In [49]:
total_positivos = VotoMesa.objects.filter(opcion__partido__isnull=False, categoria='sen').aggregate(v=Sum('votos'))['v']
total_positivos

8601410

In [8]:
from django.db.models import Avg, Sum

In [50]:
VotoMesa.objects.filter(opcion=UC, categoria='sen').aggregate(v=Sum('votos'))['v']/total_positivos

0.3533368366349238

In [51]:
VotoMesa.objects.filter(opcion=CAMBIEMOS, categoria='sen').aggregate(v=Sum('votos'))['v']/total_positivos

0.35414077459393284

Según los cálculos, **Unidad Ciudadana tiene 35.33 %** y **Cambiemos el 35.41%** para senadores. Un 0.08% de diferencia

In [54]:
total_positivos_dip = VotoMesa.objects.filter(opcion__partido__isnull=False, categoria='dip').aggregate(v=Sum('votos'))['v']
total_positivos_dip

8438373

In [55]:
VotoMesa.objects.filter(opcion=UC, categoria='dip').aggregate(v=Sum('votos'))['v']/total_positivos_dip

0.3407902210532765

In [56]:
VotoMesa.objects.filter(opcion=CAMBIEMOS, categoria='dip').aggregate(v=Sum('votos'))['v']/total_positivos_dip

0.364079070692893

Para diputados **Unidad Ciudadana tiene 34.07 %** y **Cambiemos el 36.40%**

### Mesas donde el UC sacó menos del 50% de los votos promedio a Senador en el circuito

In [3]:
from IPython.display import display_html, HTML

In [17]:
mesas = []
for circuito in Circuito.objects.all():
    promedio_uc_circuito = VotoMesa.objects.filter(mesa__circuito=circuito, opcion=UC, categoria='sen').aggregate(p=Avg('votos'))['p']
    if not promedio_uc_circuito:
        continue
    
    for mesa in Mesa.objects.filter(circuito=circuito, resultados__votos__gt=0, 
                                    resultados__votos__lte=promedio_uc_circuito/2, 
                                    resultados__opcion=UC, resultados__categoria='sen'):
        vot_uc = mesa.resultados.get(opcion=UC, categoria='sen').votos
        
        mesas.append((mesa, vot_uc, promedio_uc_circuito))


In [18]:
len(mesas)

336

In [19]:
html = '<table class="table"><tr><th>Mesa</th><th>Circuito</th><th>Computados</th><th>Votos UC</th><th>Promedio UC/Circ</th></tr>'
for l in mesas:
    mesa, vot_uc, promedio_uc_circuito = l
    html += f"<tr><td><a href='{mesa.absolute_url}'>Mesa {mesa.numero}</a></td><td> {mesa.circuito} </td><td> {mesa.computados} </td><td>{vot_uc}</td><td> {promedio_uc_circuito:.2f}</td></tr>"
html += '</table>'
    
display_html(HTML(html))

Mesa Circuito Computados Votos UC Promedio UC/Circ Mesa 00107 Circuito 0789 (011 - Berazategui) 170 39 98.13 Mesa 00267 Circuito 0789C (011 - Berazategui) 373 40 84.37 Mesa 00256 Circuito 0789C (011 - Berazategui) 394 4 84.37 Mesa 00336 Circuito 0789D (011 - Berazategui) 264 1 121.37 Mesa 00573 Circuito 0792 (011 - Berazategui) 116 15 101.40 Mesa 00526 Circuito 0792 (011 - Berazategui) 603 18 101.40 Mesa 00727 Circuito 0792D (011 - Berazategui) 312 62 125.36 Mesa 00726 Circuito 0792D (011 - Berazategui) 290 58 125.36 Mesa 00067 Circuito 0117 (010 - Arrecifes) 208 21 71.01 Mesa 00093 Circuito 0073 (007 - Bahía Blanca) 292 20 40.85 Mesa 00037 Circuito 0073 (007 - Bahía Blanca) 437 5 40.85 Mesa 00307 Circuito 0078 (007 - Bahía Blanca) 196 9 41.13 Mesa 00324 Circuito 0079 (007 - Bahía Blanca) 128 20 51.59 Mesa 00394 Circuito 0084 (007 - Bahía Blanca) 540 16 37.66 Mesa 00591 Circuito 0087 (007 - Bahía Blanca) 154 23 67.80 Mesa 00556 Circuito 0087 (007 - Bahía Blanca) 410 21 67.80 Mesa 00608 Circuito 0088 (007 - Bahía Blanca) 213 27 60.28 Mesa 00607 Circuito 0088 (007 - Bahía Blanca) 282 28 60.28 Mesa 00633 Circuito 0089 (007 - Bahía Blanca) 160 8 36.44 Mesa 00068 Circuito 0105 (009 - Baradero) 315 35 74.30 Mesa 00040 Circuito 0023 (004 - Avellaneda) 141 15 66.08 Mesa 00022 Circuito 0010 (002 - Alberti) 316 17 47.70 Mesa 00024 Circuito 0011 (002 - Alberti) 182 30 60.50 Mesa 00307 Circuito 0034 (004 - Avellaneda) 190 38 90.91 Mesa 00364 Circuito 0037 (004 - Avellaneda) 234 44 88.73 Mesa 00810 Circuito 0040 (004 - Avellaneda) 214 18 85.63 Mesa 00042 Circuito 0005 (001 - Adolfo Alsina) 124 10 21.75 Mesa 00081 Circuito 0201 (030 - Chascomús) 111 6 54.28 Mesa 00069 Circuito 0017 (003 - Almirante Brown) 287 23 46.25 Mesa 00058 Circuito 0017 (003 - Almirante Brown) 444 5 46.25 Mesa 00165 Circuito 0018A (003 - Almirante Brown) 468 27 134.58 Mesa 00354 Circuito 0019 (003 - Almirante Brown) 422 14 101.10 Mesa 00660 Circuito 0020 (003 - Almirante Brown) 231 33 80.81 Mesa 00639 Circuito 0020 (003 - Almirante Brown) 455 4 80.81 Mesa 00628 Circuito 0020 (003 - Almirante Brown) 511 25 80.81 Mesa 00903 Circuito 0021 (003 - Almirante Brown) 210 49 110.66 Mesa 01189 Circuito 0022A (003 - Almirante Brown) 113 25 104.85 Mesa 00152 Circuito 0668 (134 - Hurlingham) 218 25 68.49 Mesa 00308 Circuito 0668B (134 - Hurlingham) 449 39 97.77 Mesa 00105 Circuito 0098 (008 - Balcarce) 250 29 58.03 Mesa 00110 Circuito 0101 (008 - Balcarce) 157 10 25.00 Mesa 00049 Circuito 0046 (005 - Ayacucho) 314 24 62.44 Mesa 00152 Circuito 0062 (006 - Azul) 232 14 48.67 Mesa 00168 Circuito 0067 (006 - Azul) 138 14 63.40 Mesa 00589 Circuito 0398B (129 - J. C. Paz) 450 16 110.37 Mesa 00558 Circuito 0398B (129 - J. C. Paz) 370 31 110.37 Mesa 00106 Circuito 0189 (029 - Chacabuco) 126 20 78.73 Mesa 00085 Circuito 0254 (028 - Coronel Suárez) 204 14 51.50 Mesa 00009 Circuito 0765 (088 - Pila) 134 19 79.00 Mesa 00162 Circuito 0666 (133 - Ituzaingo) 264 46 99.83 Mesa 00246 Circuito 0666A (133 - Ituzaingo) 310 40 97.19 Mesa 00236 Circuito 0666A (133 - Ituzaingo) 366 1 97.19 Mesa 00402 Circuito 0667 (133 - Ituzaingo) 242 34 70.53 Mesa 00295 Circuito 0399A (130 - Malvinas Argentinas) 248 57 115.83 Mesa 00521 Circuito 0399E (130 - Malvinas Argentinas) 180 39 97.15 Mesa 00679 Circuito 0399F (130 - Malvinas Argentinas) 362 44 91.63 Mesa 00561 Circuito 0399F (130 - Malvinas Argentinas) 487 3 91.63 Mesa 00739 Circuito 0399G (130 - Malvinas Argentinas) 216 53 141.85 Mesa 00723 Circuito 0399G (130 - Malvinas Argentinas) 326 39 141.85 Mesa 00132 Circuito 0289 (132 - Ezeiza) 185 26 85.30 Mesa 00039 Circuito 0289 (132 - Ezeiza) 500 42 85.30 Mesa 00001 Circuito 0289 (132 - Ezeiza) 543 19 85.30 Mesa 00139 Circuito 0290 (132 - Ezeiza) 456 15 99.32 Mesa 00338 Circuito 0290B (132 - Ezeiza) 445 28 127.68 Mesa 00050 Circuito 0304 (128 - Presidente Perón) 512 48 112.31 Mesa 00127 Circuito 0304A (128 - Presidente Perón) 140 38 94.89 Mesa 00101 Circuito 0304A (128 - Presidente Perón) 450 10 94.89 Mesa 00080 C

### Mesas no computadas

In [28]:
html = '<table class="table"><tr><th>Mesa</th><th>Circuito</th></tr>'
for mesa in Mesa.objects.filter(resultados__votos=0).distinct():
    if mesa.resultados.all().aggregate(p=Avg('votos'))['p'] != 0:
        # no computadas
        continue        
    html += f"<tr><td><a href='{mesa.absolute_url}'>Mesa {mesa.numero}</a></td><td> {mesa.circuito} </td></tr>"
html += '</table>'
    
display_html(HTML(html))

Mesa Circuito Mesa 00025 Circuito 0668 (134 - Hurlingham) Mesa 00953 Circuito 0022 (003 - Almirante Brown) Mesa 01037 Circuito 0022 (003 - Almirante Brown) Mesa 09037 Circuito 0022 (003 - Almirante Brown) Mesa 09035 Circuito 0022 (003 - Almirante Brown) Mesa 09036 Circuito 0022 (003 - Almirante Brown) Mesa 09012 Circuito 0667 (133 - Ituzaingo) Mesa 09010 Circuito 0667 (133 - Ituzaingo) Mesa 09008 Circuito 0667 (133 - Ituzaingo) Mesa 09006 Circuito 0667 (133 - Ituzaingo) Mesa 09001 Circuito 0667 (133 - Ituzaingo) Mesa 09003 Circuito 0667 (133 - Ituzaingo) Mesa 09002 Circuito 0667 (133 - Ituzaingo) Mesa 09014 Circuito 0666A (133 - Ituzaingo) Mesa 09050 Circuito 0022A (003 - Almirante Brown) Mesa 09049 Circuito 0022A (003 - Almirante Brown) Mesa 09048 Circuito 0022A (003 - Almirante Brown) Mesa 09046 Circuito 0022A (003 - Almirante Brown) Mesa 09047 Circuito 0022A (003 - Almirante Brown) Mesa 09045 Circuito 0022A (003 - Almirante Brown) Mesa 09044 Circuito 0022A (003 - Almirante Brown) Mesa 01180 Circuito 0022A (003 - Almirante Brown) Mesa 09032 Circuito 0022 (003 - Almirante Brown) Mesa 09016 Circuito 0668 (134 - Hurlingham) Mesa 09031 Circuito 0022 (003 - Almirante Brown) Mesa 09030 Circuito 0022 (003 - Almirante Brown) Mesa 09029 Circuito 0022 (003 - Almirante Brown) Mesa 09024 Circuito 0022 (003 - Almirante Brown) Mesa 09012 Circuito 0668 (134 - Hurlingham) Mesa 09013 Circuito 0668 (134 - Hurlingham) Mesa 09010 Circuito 0668 (134 - Hurlingham) Mesa 09011 Circuito 0668 (134 - Hurlingham) Mesa 09005 Circuito 0668 (134 - Hurlingham) Mesa 09006 Circuito 0668 (134 - Hurlingham) Mesa 09007 Circuito 0668 (134 - Hurlingham) Mesa 09004 Circuito 0668 (134 - Hurlingham) Mesa 00347 Circuito 0668B (134 - Hurlingham) Mesa 00691 Circuito 0399G (130 - Malvinas Argentinas) Mesa 09051 Circuito 0022B (003 - Almirante Brown) Mesa 09052 Circuito 0022B (003 - Almirante Brown) Mesa 00288 Circuito 0667 (133 - Ituzaingo) Mesa 00284 Circuito 0667 (133 - Ituzaingo) Mesa 09003 Circuito 0668 (134 - Hurlingham) Mesa 09002 Circuito 0668 (134 - Hurlingham) Mesa 09001 Circuito 0668 (134 - Hurlingham) Mesa 01017 Circuito 0022 (003 - Almirante Brown) Mesa 01211 Circuito 0022B (003 - Almirante Brown) Mesa 00269 Circuito 0667 (133 - Ituzaingo) Mesa 00205 Circuito 0668A (134 - Hurlingham) Mesa 00309 Circuito 0668B (134 - Hurlingham) Mesa 01112 Circuito 0022A (003 - Almirante Brown) Mesa 01110 Circuito 0022A (003 - Almirante Brown) Mesa 00918 Circuito 0021A (003 - Almirante Brown) Mesa 01190 Circuito 0022B (003 - Almirante Brown) Mesa 00101 Circuito 0668 (134 - Hurlingham) Mesa 00073 Circuito 0668 (134 - Hurlingham) Mesa 00059 Circuito 0668 (134 - Hurlingham) Mesa 01044 Circuito 0022A (003 - Almirante Brown) Mesa 00034 Circuito 0668 (134 - Hurlingham) Mesa 00026 Circuito 0668 (134 - Hurlingham) Mesa 09001 Circuito 0207L (135 - Lezama) Mesa 09001 Circuito 0361 (127 - F. Ameghino) Mesa 00091 Circuito 0289 (132 - Ezeiza) Mesa 00659 Circuito 0399F (130 - Malvinas Argentinas) Mesa 09010 Circuito 0399 (130 - Malvinas Argentinas) Mesa 09009 Circuito 0399 (130 - Malvinas Argentinas) Mesa 09008 Circuito 0399 (130 - Malvinas Argentinas) Mesa 09006 Circuito 0399 (130 - Malvinas Argentinas) Mesa 09005 Circuito 0399 (130 - Malvinas Argentinas) Mesa 09007 Circuito 0399 (130 - Malvinas Argentinas) Mesa 09004 Circuito 0399 (130 - Malvinas Argentinas) Mesa 09003 Circuito 0399 (130 - Malvinas Argentinas) Mesa 09001 Circuito 0399 (130 - Malvinas Argentinas) Mesa 09002 Circuito 0399 (130 - Malvinas Argentinas) Mesa 00172 Circuito 0399 (130 - Malvinas Argentinas) Mesa 00168 Circuito 0399 (130 - Malvinas Argentinas) Mesa 00139 Circuito 0399 (130 - Malvinas Argentinas) Mesa 00678 Circuito 0399F (130 - Malvinas Argentinas) Mesa 09003 Circuito 0289 (132 - Ezeiza) Mesa 09001 Circuito 0289 (132 - Ezeiza) Mesa 00106 Circuito 0399 (130 - Malvinas Argentinas) Mesa 00110 Circuito 0289 (132 - Ezeiza) Mesa 00105 Circuito 0289 (132 - Ezeiza) Mesa 00077 Circuito 0399 (130 - Malvinas Argentinas) Mes

### Mesas para los que UC sacó 0 votos, para senador o diputado

In [27]:
html = '<table class="table"><tr><th>Mesa</th><th>Circuito</th><th>Computados</th><th>Promedio UC/Circ</th></tr>'
for mesa in Mesa.objects.filter(resultados__votos=0, resultados__opcion=UC).distinct():
    if mesa.resultados.all().aggregate(p=Avg('votos'))['p'] == 0:
        # no computadas
        continue
    
    promedio_circuito = VotoMesa.objects.filter(mesa__circuito=mesa.circuito, opcion=UC, categoria='sen').aggregate(p=Avg('votos'))['p']
    
    html += f"<tr><td><a href='{mesa.absolute_url}'>Mesa {mesa.numero}</a></td><td> {mesa.circuito} </td><td> {mesa.computados} </td><td> {promedio_circuito:.2f} </td></tr>"
html += '</table>'
    
display_html(HTML(html))

Mesa Circuito Computados Promedio UC/Circ Mesa 09011 Circuito 0667 (133 - Ituzaingo) 4 70.53 Mesa 09009 Circuito 0667 (133 - Ituzaingo) 2 70.53 Mesa 09007 Circuito 0667 (133 - Ituzaingo) 12 70.53 Mesa 09005 Circuito 0667 (133 - Ituzaingo) 4 70.53 Mesa 09004 Circuito 0667 (133 - Ituzaingo) 6 70.53 Mesa 00396 Circuito 0667 (133 - Ituzaingo) 562 70.53 Mesa 09013 Circuito 0666A (133 - Ituzaingo) 2 97.19 Mesa 09033 Circuito 0022 (003 - Almirante Brown) 12 95.11 Mesa 09017 Circuito 0668 (134 - Hurlingham) 4 68.49 Mesa 09026 Circuito 0022 (003 - Almirante Brown) 2 95.11 Mesa 09028 Circuito 0022 (003 - Almirante Brown) 4 95.11 Mesa 09025 Circuito 0022 (003 - Almirante Brown) 2 95.11 Mesa 09023 Circuito 0022 (003 - Almirante Brown) 4 95.11 Mesa 00306 Circuito 0667 (133 - Ituzaingo) 148 70.53 Mesa 09014 Circuito 0668 (134 - Hurlingham) 10 68.49 Mesa 09015 Circuito 0668 (134 - Hurlingham) 10 68.49 Mesa 09008 Circuito 0668 (134 - Hurlingham) 14 68.49 Mesa 09009 Circuito 0668 (134 - Hurlingham) 4 68.49 Mesa 01219 Circuito 0022B (003 - Almirante Brown) 198 113.02 Mesa 01208 Circuito 0022B (003 - Almirante Brown) 194 113.02 Mesa 00202 Circuito 0668A (134 - Hurlingham) 305 116.20 Mesa 00964 Circuito 0022 (003 - Almirante Brown) 253 95.11 Mesa 00174 Circuito 0225 (031 - Chivilcoy) 22 0.00 Mesa 09002 Circuito 0289 (132 - Ezeiza) 30 85.30 Mesa 09006 Circuito 0289 (132 - Ezeiza) 16 85.30 Mesa 09004 Circuito 0289 (132 - Ezeiza) 10 85.30 Mesa 09005 Circuito 0289 (132 - Ezeiza) 2 85.30 Mesa 09014 Circuito 0399A (130 - Malvinas Argentinas) 2 115.83 Mesa 09015 Circuito 0399B (130 - Malvinas Argentinas) 2 100.37 Mesa 00617 Circuito 0399F (130 - Malvinas Argentinas) 363 91.63 Mesa 09016 Circuito 0290 (132 - Ezeiza) 14 99.32 Mesa 09014 Circuito 0290 (132 - Ezeiza) 12 99.32 Mesa 09012 Circuito 0290 (132 - Ezeiza) 2 99.32 Mesa 00358 Circuito 0399B (130 - Malvinas Argentinas) 174 100.37 Mesa 00041 Circuito 0666 (133 - Ituzaingo) 392 99.83 Mesa 09011 Circuito 0290 (132 - Ezeiza) 14 99.32 Mesa 09008 Circuito 0290 (132 - Ezeiza) 14 99.32 Mesa 09010 Circuito 0290 (132 - Ezeiza) 38 99.32 Mesa 09007 Circuito 0290 (132 - Ezeiza) 12 99.32 Mesa 09021 Circuito 0290A (132 - Ezeiza) 6 100.60 Mesa 09019 Circuito 0290A (132 - Ezeiza) 2 100.60 Mesa 09020 Circuito 0290A (132 - Ezeiza) 8 100.60 Mesa 00577 Circuito 0399F (130 - Malvinas Argentinas) 515 91.63 Mesa 09004 Circuito 0304A (128 - Presidente Perón) 2 94.89 Mesa 09003 Circuito 0304A (128 - Presidente Perón) 6 94.89 Mesa 00502 Circuito 0399E (130 - Malvinas Argentinas) 510 97.15 Mesa 00001 Circuito 0399 (130 - Malvinas Argentinas) 201 116.97 Mesa 00553 Circuito 0399F (130 - Malvinas Argentinas) 203 91.63 Mesa 00198 Circuito 0399A (130 - Malvinas Argentinas) 285 115.83 Mesa 00246 Circuito 0290A (132 - Ezeiza) 94 100.60 Mesa 00081 Circuito 0304A (128 - Presidente Perón) 342 94.89 Mesa 00235 Circuito 0290A (132 - Ezeiza) 102 100.60 Mesa 00090 Circuito 0257 (028 - Coronel Suárez) 4 0.00 Mesa 09001 Circuito 0613 (131 - Punta Indio) 14 79.18 Mesa 00024 Circuito 0252 (028 - Coronel Suárez) 558 57.29 Mesa 00086 Circuito 0255 (028 - Coronel Suárez) 50 0.00 Mesa 00091 Circuito 0060 (006 - Azul) 449 65.92 Mesa 09024 Circuito 0398 (129 - J. C. Paz) 8 92.16 Mesa 09023 Circuito 0398 (129 - J. C. Paz) 28 92.16 Mesa 09020 Circuito 0398 (129 - J. C. Paz) 4 92.16 Mesa 09019 Circuito 0398 (129 - J. C. Paz) 8 92.16 Mesa 09022 Circuito 0398 (129 - J. C. Paz) 6 92.16 Mesa 09017 Circuito 0398 (129 - J. C. Paz) 8 92.16 Mesa 09016 Circuito 0398 (129 - J. C. Paz) 6 92.16 Mesa 09011 Circuito 0398 (129 - J. C. Paz) 4 92.16 Mesa 09010 Circuito 0398 (129 - J. C. Paz) 6 92.16 Mesa 09006 Circuito 0398 (129 - J. C. Paz) 8 92.16 Mesa 09005 Circuito 0398 (129 - J. C. Paz) 14 92.16 Mesa 09003 Circuito 0398 (129 - J. C. Paz) 14 92.16 Mesa 09002 Circuito 0398 (129 - J. C. Paz) 4 92.16 Mesa 09004 Circuito 0398 (129 - J. C. Paz) 6 92.16 Mesa 00202 Circuito 0398 (129 - J. C. Paz) 308 92.16 Mesa 00175 Circuito 0398 (129 - J. C. Paz) 423 92.16 Mesa 00038 Circ

### Mesas para los que Cambiemos sacó 70% o más del promedio de su circuito

In [23]:
mesas = []
for circuito in Circuito.objects.all():
    promedio_circuito = VotoMesa.objects.filter(mesa__circuito=circuito, opcion=CAMBIEMOS, categoria='sen').aggregate(p=Avg('votos'))['p']
    if not promedio_circuito:
        continue
    
    for mesa in Mesa.objects.filter(circuito=circuito, resultados__votos__gt=0, 
                                    resultados__votos__gte=promedio_circuito*1.7, 
                                    resultados__opcion=CAMBIEMOS, resultados__categoria='sen'):
        vot = mesa.resultados.get(opcion=CAMBIEMOS, categoria='sen').votos
        
        mesas.append((mesa, vot, promedio_circuito))


In [24]:
html = '<table class="table"><tr><th>Mesa</th><th>Circuito</th><th>Computados</th><th>Votos CAMBIEMOS</th><th>Promedio CAMBIEMOS/Circ</th></tr>'
for l in mesas:
    mesa, vot, promedio_circuito = l
    html += f"<tr><td><a href='{mesa.absolute_url}'>Mesa {mesa.numero}</a></td><td> {mesa.circuito} </td><td> {mesa.computados} </td><td>{vot}</td><td> {promedio_circuito:.2f}</td></tr>"
html += '</table>'
    
display_html(HTML(html))

Mesa Circuito Computados Votos CAMBIEMOS Promedio CAMBIEMOS/Circ Mesa 00093 Circuito 0789 (011 - Berazategui) 563 89 48.60 Mesa 00079 Circuito 0789 (011 - Berazategui) 550 86 48.60 Mesa 00061 Circuito 0789 (011 - Berazategui) 570 88 48.60 Mesa 00081 Circuito 0110 (009 - Baradero) 188 38 19.00 Mesa 00198 Circuito 0031 (004 - Avellaneda) 580 111 55.00 Mesa 00200 Circuito 0031 (004 - Avellaneda) 538 103 55.00 Mesa 00199 Circuito 0031 (004 - Avellaneda) 546 98 55.00 Mesa 00024 Circuito 0011 (002 - Alberti) 182 21 10.50 Mesa 00160 Circuito 0218 (031 - Chivilcoy) 278 50 25.00 Mesa 00040 Circuito 0668 (134 - Hurlingham) 606 193 94.43 Mesa 00058 Circuito 0398 (129 - J. C. Paz) 572 102 59.51 Mesa 00390 Circuito 0398A (129 - J. C. Paz) 588 89 42.22 Mesa 00262 Circuito 0398A (129 - J. C. Paz) 600 97 42.22 Mesa 00102 Circuito 0289 (132 - Ezeiza) 578 150 80.06 Mesa 00084 Circuito 0985 (121 - Veinticinco de Mayo) 524 111 62.50 Mesa 00004 Circuito 0951 (116 - Tordillo) 573 109 57.29 Mesa 00003 Circuito 0951 (116 - Tordillo) 570 100 57.29 Mesa 00153 Circuito 0975B (117 - Tres Arroyos) 301 60 30.00 Mesa 00218 Circuito 0534 (113 - Tigre) 548 175 94.84 Mesa 00758 Circuito 0536 (113 - Tigre) 566 109 60.06 Mesa 00371 Circuito 0892 (106 - San Isidro) 652 191 103.23 Mesa 00315 Circuito 0905 (107 - San Nicolás) 623 97 51.93 Mesa 00016 Circuito 0872 (105 - San Fernando) 562 158 88.63 Mesa 00008 Circuito 0872 (105 - San Fernando) 554 154 88.63 Mesa 00013 Circuito 0872 (105 - San Fernando) 554 151 88.63 Mesa 00292 Circuito 0879A (105 - San Fernando) 626 98 50.21 Mesa 00038 Circuito 0339A (090 - Pinamar) 514 97 51.16 Mesa 00157 Circuito 0768 (089 - Pilar) 573 145 82.04 Mesa 00119 Circuito 0768 (089 - Pilar) 574 144 82.04 Mesa 00261 Circuito 0770 (089 - Pilar) 590 126 62.29 Mesa 00257 Circuito 0770 (089 - Pilar) 548 107 62.29 Mesa 00495 Circuito 0786 (092 - Quilmes) 550 132 77.47 Mesa 00697 Circuito 0788 (092 - Quilmes) 586 75 42.37 Mesa 00967 Circuito 0788B (092 - Quilmes) 569 110 60.22 Mesa 00921 Circuito 0788B (092 - Quilmes) 398 105 60.22 Mesa 00999 Circuito 0788C (092 - Quilmes) 633 98 35.49 Mesa 01243 Circuito 0791B (092 - Quilmes) 589 125 73.20 Mesa 00779 Circuito 0654 (077 - Merlo) 558 77 40.75 Mesa 00228 Circuito 0600 (071 - Luján) 508 113 62.00 Mesa 00140 Circuito 0583 (070 - Lomas de Zamora) 530 72 41.50 Mesa 00253 Circuito 0583 (070 - Lomas de Zamora) 525 77 41.50 Mesa 00152 Circuito 0583 (070 - Lomas de Zamora) 592 71 41.50 Mesa 00137 Circuito 0583 (070 - Lomas de Zamora) 492 71 41.50 Mesa 00879 Circuito 0587 (070 - Lomas de Zamora) 521 126 70.92 Mesa 00686 Circuito 0587 (070 - Lomas de Zamora) 568 121 70.92 Mesa 00034 Circuito 0622 (074 - Mar Chiquita) 544 125 65.83 Mesa 00052 Circuito 0468 (063 - La Plata) 524 135 74.90 Mesa 00236 Circuito 0486 (063 - La Plata) 560 138 72.33 Mesa 01269 Circuito 0505B (063 - La Plata) 534 67 37.18 Mesa 01266 Circuito 0505B (063 - La Plata) 514 68 37.18 Mesa 01429 Circuito 0508F (063 - La Plata) 452 93 52.86 Mesa 01435 Circuito 0509 (063 - La Plata) 570 94 52.63 Mesa 00133 Circuito 0626 (061 - La Matanza) 562 148 82.47 Mesa 00073 Circuito 0626 (061 - La Matanza) 562 141 82.47 Mesa 00682 Circuito 0629A (061 - La Matanza) 570 85 47.44 Mesa 00590 Circuito 0629A (061 - La Matanza) 552 82 47.44 Mesa 01360 Circuito 0632A (061 - La Matanza) 594 107 60.93 Mesa 01332 Circuito 0632A (061 - La Matanza) 576 108 60.93 Mesa 01444 Circuito 0633 (061 - La Matanza) 520 123 70.97 Mesa 01416 Circuito 0633 (061 - La Matanza) 533 121 70.97 Mesa 01394 Circuito 0633 (061 - La Matanza) 548 124 70.97 Mesa 01855 Circuito 0635 (061 - La Matanza) 700 56 32.32 Mesa 01852 Circuito 0635 (061 - La Matanza) 540 61 32.32 Mesa 01869 Circuito 0635 (061 - La Matanza) 514 63 32.32 Mesa 01844 Circuito 0635 (061 - La Matanza) 542 57 32.32 Mesa 01828 Circuito 0635 (061 - La Matanza) 534 55 32.32 Mesa 01818 Circuito 0635 (061 - La Matanza) 573 61 32.32 Mesa 01745 Circuito 0635 (061 - La Matanza) 529 55 32.32 Mesa 01688 Circuito 0635 (061 - La Matanza

### Mesas con más de 30% de corte de boleta para UC

In [34]:
html = '<table class="table"><tr><th>Mesa</th><th>Circuito</th><th>Computados</th><th>Votos Sen</th><th>Votos Dip</th><th>% diferencia</th></tr>'
for mesa in Mesa.objects.all():
    vot_dip = mesa.resultados.filter(opcion=UC, categoria='dip').first()
    vot_sen = mesa.resultados.filter(opcion=UC, categoria='sen').first()
    if vot_dip and vot_sen and vot_dip.votos and vot_sen.votos:
        vot_dip, vot_sen = vot_dip.votos, vot_sen.votos
        diferencia =  max(vot_dip, vot_sen)/min(vot_dip, vot_sen) 
        if diferencia > 1.3:
            diferencia = (diferencia - 1) * 100
            html += f"<tr><td><a href='{mesa.absolute_url}'>Mesa {mesa.numero}</a></td><td> {mesa.circuito} </td><td> {mesa.computados} </td><td> {vot_sen} </td><td> {vot_dip} </td><td> {diferencia:.1f}% </td></tr>"
html += '</table>'
    
display_html(HTML(html))


Mesa Circuito Computados Votos Sen Votos Dip % diferencia Mesa 00924 Circuito 0021A (003 - Almirante Brown) 604 107 180 68.2% Mesa 00308 Circuito 0668B (134 - Hurlingham) 449 39 85 117.9% Mesa 00099 Circuito 0258B (028 - Coronel Suárez) 564 96 71 35.2% Mesa 00100 Circuito 0258B (028 - Coronel Suárez) 582 77 49 57.1% Mesa 00102 Circuito 0258B (028 - Coronel Suárez) 568 86 64 34.4% Mesa 00103 Circuito 0258B (028 - Coronel Suárez) 434 73 46 58.7% Mesa 01006 Circuito 0022 (003 - Almirante Brown) 544 116 88 31.8% Mesa 00779 Circuito 0021 (003 - Almirante Brown) 586 144 107 34.6% Mesa 00226 Circuito 0668A (134 - Hurlingham) 432 105 44 138.6% Mesa 00335 Circuito 0667 (133 - Ituzaingo) 493 61 5 1120.0% Mesa 00001 Circuito 0361 (127 - F. Ameghino) 592 38 20 90.0% Mesa 00002 Circuito 0361 (127 - F. Ameghino) 574 29 16 81.2% Mesa 00003 Circuito 0361 (127 - F. Ameghino) 548 34 19 78.9% Mesa 00004 Circuito 0361 (127 - F. Ameghino) 590 41 29 41.4% Mesa 00005 Circuito 0361 (127 - F. Ameghino) 576 39 21 85.7% Mesa 00006 Circuito 0361 (127 - F. Ameghino) 560 30 15 100.0% Mesa 00007 Circuito 0361 (127 - F. Ameghino) 568 31 18 72.2% Mesa 00009 Circuito 0361 (127 - F. Ameghino) 590 33 18 83.3% Mesa 00023 Circuito 0362 (127 - F. Ameghino) 580 43 26 65.4% Mesa 00024 Circuito 0362 (127 - F. Ameghino) 492 32 14 128.6% Mesa 00010 Circuito 0361 (127 - F. Ameghino) 582 27 10 170.0% Mesa 00011 Circuito 0361 (127 - F. Ameghino) 570 34 19 78.9% Mesa 00013 Circuito 0361 (127 - F. Ameghino) 546 33 14 135.7% Mesa 00015 Circuito 0361 (127 - F. Ameghino) 582 23 14 64.3% Mesa 00016 Circuito 0361 (127 - F. Ameghino) 582 41 20 105.0% Mesa 00017 Circuito 0361 (127 - F. Ameghino) 582 40 18 122.2% Mesa 00018 Circuito 0361 (127 - F. Ameghino) 580 24 15 60.0% Mesa 00019 Circuito 0361 (127 - F. Ameghino) 574 32 17 88.2% Mesa 00020 Circuito 0361 (127 - F. Ameghino) 580 31 17 82.4% Mesa 00021 Circuito 0361 (127 - F. Ameghino) 190 18 7 157.1% Mesa 00308 Circuito 0290B (132 - Ezeiza) 540 98 73 34.2% Mesa 00310 Circuito 0290B (132 - Ezeiza) 499 148 92 60.9% Mesa 00234 Circuito 0290A (132 - Ezeiza) 520 100 58 72.4% Mesa 00139 Circuito 0290 (132 - Ezeiza) 456 15 114 660.0% Mesa 00001 Circuito 0289 (132 - Ezeiza) 543 19 94 394.7% Mesa 00338 Circuito 0290B (132 - Ezeiza) 445 28 123 339.3% Mesa 00154 Circuito 0290 (132 - Ezeiza) 528 113 55 105.5% Mesa 00101 Circuito 0304A (128 - Presidente Perón) 450 10 93 830.0% Mesa 00561 Circuito 0399F (130 - Malvinas Argentinas) 487 3 92 2966.7% Mesa 00217 Circuito 0399A (130 - Malvinas Argentinas) 526 104 76 36.8% Mesa 00039 Circuito 0289 (132 - Ezeiza) 500 42 89 111.9% Mesa 00050 Circuito 0304 (128 - Presidente Perón) 512 48 91 89.6% Mesa 00053 Circuito 0289 (132 - Ezeiza) 522 103 57 80.7% Mesa 00093 Circuito 0258A (028 - Coronel Suárez) 628 73 47 55.3% Mesa 00095 Circuito 0258A (028 - Coronel Suárez) 552 75 53 41.5% Mesa 00096 Circuito 0258A (028 - Coronel Suárez) 576 72 48 50.0% Mesa 00091 Circuito 0258 (028 - Coronel Suárez) 322 29 22 31.8% Mesa 00067 Circuito 0253 (028 - Coronel Suárez) 566 87 56 55.4% Mesa 00068 Circuito 0253 (028 - Coronel Suárez) 538 63 38 65.8% Mesa 00069 Circuito 0253 (028 - Coronel Suárez) 523 77 45 71.1% Mesa 00070 Circuito 0253 (028 - Coronel Suárez) 580 99 59 67.8% Mesa 00071 Circuito 0253 (028 - Coronel Suárez) 444 47 28 67.9% Mesa 00001 Circuito 0252 (028 - Coronel Suárez) 556 62 42 47.6% Mesa 00003 Circuito 0252 (028 - Coronel Suárez) 574 57 42 35.7% Mesa 00004 Circuito 0252 (028 - Coronel Suárez) 508 62 41 51.2% Mesa 00005 Circuito 0252 (028 - Coronel Suárez) 297 54 30 80.0% Mesa 00006 Circuito 0252 (028 - Coronel Suárez) 499 62 44 40.9% Mesa 00007 Circuito 0252 (028 - Coronel Suárez) 542 64 45 42.2% Mesa 00009 Circuito 0252 (028 - Coronel Suárez) 542 65 45 44.4% Mesa 00010 Circuito 0252 (028 - Coronel Suárez) 549 54 41 31.7% Mesa 00011 Circuito 0252 (028 - Coronel Suárez) 552 79 52 51.9% Mesa 00015 Circuito 0252 (028 - Coronel Suárez) 467 50 38 31.6% Mesa 00016 Circuito 0252 (028 - Coronel Suárez) 554 58 4